[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enolan/jax-torch-bench/blob/master/JAX_vs_PyTorch_transformer_LM_benchmark.ipynb)

In [ ]:
# Config is tuned for a P100, a 16GB GPU. You'll have to reduce the batch sizes to get
# the models running if Colab gives you something smaller.
!nvidia-smi

# Common setup

In [ ]:
!wget http://mattmahoney.net/dc/enwik9.zip
!unzip enwik9.zip

In [ ]:
!pip install optax flax
!pip install -U numpy

In [ ]:
!git clone https://github.com/enolan/jax-torch-bench.git

In [ ]:
%cd jax-torch-bench

In [10]:
from config import ModelConfig

enwik9 = "/content/enwik9"

# This is the highest batch size PyTorch can handle, the JAX model can do 79
cfg = ModelConfig(seq_len=256, n_layers=12, d_model=512, num_heads=8, ff_dim=3072, dropout=0.1, batch_size=63, learning_rate=1e-3)

# JAX model

In [3]:
from jax_model import *

In [4]:
# Set up the model
params, model, optimizer, opt_state, sample = setup_all(cfg)

In [ ]:
# This will run the training loop indefinitely. Hit the stop button to abort.
params, opt_state = train_loop(model, optimizer, opt_state, params, cfg, enwik9)

In [ ]:
sample(params, "'''Star Trek'''")

# PyTorch model

Since JAX preallocates all GPU memory, you'll need to restart the runtime (Runtime -> Restart runtime) to try the PyTorch model. Then rerun the config setup cell before running the ones below.

In [3]:
from pytorch_model import *

In [4]:
lm = LM(cfg).cuda()

In [ ]:
train_loop(lm, cfg, enwik9)

In [ ]:
lm.sample("'''Star Trek'''")